In [3]:
import geopandas as gpd
import pandas as pd
import numpy as np
import glob
from matplotlib import pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel,WhiteKernel
from sklearn.model_selection import train_test_split
from shapely.geometry import Point
import urllib
from zipfile import ZipFile
%matplotlib inline

### Narrow the crime type and location

In [4]:
path = '/Users/hemingyi/Dropbox/MLCProjectData/AggregatedData/'

In [ ]:
urllib.request.urlretrieve('https://data.cityofnewyork.us/api/views/5uac-w243/rows.csv?accessType=DOWNLOAD',path+'OriginCrimeData.csv')


In [2]:

df = pd.read_csv(path+'OriginCrimeData.csv')
steet_crime = ['GRAND LARCENY', 'ROBBERY', 'FELONY ASSAULT', 'RAPE', 'BURGLARY', 'GRAND LARCENY OF MOTOR VEHICLE', 
               'HOMICIDE-NEGLIGENT,UNCLASSIFIE', 'KIDNAPPING & RELATED OFFENSES', 'KIDNAPPING','HOMICIDE-NEGLIGENT-VEHICLE']
df = df[df.OFNS_DESC.isin(steet_crime)]
df = df[(df['PREM_TYP_DESC']=='STREET')|(df['PREM_TYP_DESC']=='TUNNEL')|(df['PREM_TYP_DESC']=='BRIDGE')]
df.to_csv(path+'Street_related_crime.csv',index=False)

### spatial join crime to census tract

In [ ]:
urllib.request,urlretrieve('https://data.cityofnewyork.us/api/geospatial/fxpq-c8ku?method=export&format=GeoJSON',path+'CensusTract.geojson')


In [98]:
df = pd.read_csv(path+'Street_related_crime.csv')
df = df[['Longitude','Latitude']]
shp = gpd.GeoDataFrame.from_file(path+'CensusTract.geojson')

In [100]:
geometry = [Point(xy) for xy in zip(df.Longitude, df.Latitude)]
crs = {'init': 'epsg:4326'}
gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
shp = shp.to_crs(crs = {'init': 'epsg:4326'})

In [102]:
join = gpd.sjoin(shp,gdf,how="inner", op='intersects')

In [103]:
joinCount = join.groupby(['POLY_ID']).count()

In [105]:
joinCount['POLY_ID'] = joinCount.index
joinCount['Felony'] = joinCount['Avg_score']

In [106]:
joinCount = joinCount[['Felony']]

In [107]:
shp = shp.merge(joinCount, on='POLY_ID')

In [109]:
shp.to_csv(path+'CensusTractStreetCrime.csv')

### add image segmentation result

In [182]:
CrimeImage = pd.read_csv(path+"_pspnet_front.csv")
CrimePoint = pd.read_csv(path+'Street_related_crime.csv')


In [183]:
CrimeImage['index'] = CrimeImage['FID'].apply(lambda x: x.replace('Front',''))
del CrimeImage['FID']

In [184]:
CrimePoint = CrimePoint[['Latitude','Longitude']]
CrimePoint['index'] = CrimePoint.index.astype('str')

In [186]:
CrimeImagePoint = CrimeImage.merge(CrimePoint,on='index')

In [193]:
Random = pd.read_csv(path+'RandomPointImage.csv')

In [194]:
geometry = [Point(xy) for xy in zip(CrimeImagePoint.Longitude, CrimeImagePoint.Latitude)]
crs = {'init': 'epsg:4326'}
CrimeImagePointshp = gpd.GeoDataFrame(CrimeImagePoint, crs=crs, geometry=geometry)
geometry = [Point(xy) for xy in zip(Random.Lng, Random.Lat)]
Randomshp= gpd.GeoDataFrame(Random, crs=crs, geometry=geometry)

In [199]:
Randomshp = Randomshp[['building', 'sky', 'road', 'sidewalk', 'person', 'car', 'awning',
       'bicycle', 'tree', 'grass', 'plant', 'signboard', 'streetlight',
       'ashcan', 'wall', 'railing', 'van', 'fence', 'earth', 'mountain',
       'ceiling', 'minibike', 'bridge', 'skyscraper', 'water', 'windowpane',
       'column', 'sculpture', 'lamp', 'fountain', 'chair', 'bulletin board',
       'geometry']]
CrimeImagePointshp = CrimeImagePointshp[['building', 'sky', 'road', 'sidewalk', 'person', 'car', 'awning',
       'bicycle', 'tree', 'grass', 'plant', 'signboard', 'streetlight',
       'ashcan', 'wall', 'railing', 'van', 'fence', 'earth', 'mountain',
       'ceiling', 'minibike', 'bridge', 'skyscraper', 'water', 'windowpane',
       'column', 'sculpture', 'lamp', 'fountain', 'chair', 'bulletin board',
       'geometry']]

In [228]:
Points = pd.concat([Randomshp, CrimeImagePointshp])
Points.to_csv(path+'AllSamplePoints.csv')

In [211]:
CensusTract = path+'CensusTract.geojson'
polygon = gpd.GeoDataFrame.from_file(CensusTract)
polygon = polygon[polygon['boro_code']=='1']
join = gpd.sjoin(Points,polygon,how="inner", op='intersects')

In [212]:
joinCount = join.groupby(['ct2010']).mean()

In [216]:
df = pd.read_csv(path+'Street_related_crime.csv')
df = df[['Longitude','Latitude']]
shp = gpd.GeoDataFrame.from_file(CensusTract)
shp = shp[shp['boro_code']=='1']
geometry = [Point(xy) for xy in zip(df.Longitude, df.Latitude)]
crs = {'init': 'epsg:4326'}
gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
join = gpd.sjoin(shp,gdf,how="inner", op='intersects')

In [220]:
joinCrimeCount = join.groupby(['ct2010']).count()

In [222]:
joinCrimeCount['Felony'] = joinCrimeCount['shape_area']
joinCrimeCount = joinCrimeCount[['CrimeCount']]

In [225]:
joinCount = joinCount.merge(joinCrimeCount,on='ct2010')

In [232]:
joinCount['tract'] = joinCount.index.astype('str')

In [233]:
joinCount.to_csv(path+"StreetAvgPspNet.csv",index=False)

In [ ]:
urllib.request.urlretrieve('https://www.kaggle.com/muonneutrino/new-york-city-census-data/downloads/new-york-city-census-data.zip/2',path+'DemographicsData.zip')


In [288]:
df = pd.read_csv(path+"demographic_census.csv")
df.dropna(inplace=True)
shp = pd.read_csv(path+"StreetAvgPspNet.csv")
shp['Felony'] = shp['CrimeCount']
shp['plants'] = shp['tree']+shp['grass']+shp['plant']
Class = pd.read_csv(path+"CensusTractSVI.csv")

In [289]:
Class = Class[['yCut','tract']]

In [290]:
mergedf = shp.merge(df, left_on='tract', right_on='ct2010')
mergedf = mergedf.merge(Class,left_on='tract', right_on='tract')
mergedf['class'] = mergedf['yCut']

In [291]:
del mergedf['Unnamed: 0']
del mergedf['boro_code']
del mergedf['boro_name']
del mergedf['shape_area']
del mergedf['shape_leng']
del mergedf['index_right']
del mergedf['geometry']
del mergedf['yCut']
del mergedf['ct2010']

In [293]:
mergedf.to_csv(path+"/merge.csv")